In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from evo import *

params = {
    'd_model': 512,
    'nhead': 8,
    'dim_feedforward': 2048,
    'dropout': 0.1,
    'encoder_layers': 3,
    'decoder_layers': 3,
    'max_len': 64
}

evolver = Evolver(**params)
ps_evolver = PointerStyleEvolver(**params)

In [ ]:
from data import *
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_loader = supervised_loader(
    path='data/ud/ud_train_3.0.jsonl',
    max_len=10,
    tokenizer=tokenizer,
    batch_size=4,
    cache_prefix=None,
    all_tokens=True,
    limit=20,
    sampler=StratifiedInfiniteSampler
)

eval_loader = unsupervised_loader(
    path='data/toy/toy.jsonl',
    max_len=10,
    tokenizer=tokenizer,
    batch_size=4,
    sampler=StratifiedInfiniteSampler
)

In [ ]:
from run import apply_edits

traj_input_ids, _, traj_edit_tgts, _ = next(iter(train_loader))

apply_edits(traj_input_ids[:, 0], tuple(map(lambda x: x[:, 0], traj_edit_tgts)))

In [ ]:
from torch.optim import AdamW

kwargs = {
    'train_loader': train_loader,
    'eval_loader': eval_loader,
    'train_steps': 1,
    'eval_steps': 2,
    'grad_accum_steps': 1,
    'clip_gradients': False,
    'checkpoint_at': 20,
    'eval_at': 1
}

print('STARTING REGULAR EVOLVER')
train_evolver(evolver, AdamW(evolver.parameters(), lr=3e-4), None, **kwargs)

print('STARTING PS EVOLVER')
train_evolver(ps_evolver, AdamW(ps_evolver.parameters(), lr=3e-4), None, **kwargs)

## multihead pointer

In [ ]:
import torch
from trans import MultiheadPointer

pointer = MultiheadPointer(512, 8)

mem = torch.randn(3, 10, 512)
tgt = torch.randn(3, 5, 512)
src_pad_mask = torch.full((3, 10), True)
src_pad_mask[:, :7] = False

idx_weights = pointer(tgt, mem, key_padding_mask=src_pad_mask)
idx_weights

## regressions

```
python evo.py --config=configs/toy/sup-toy.json
python evo.py --config=configs/toy/sup-toy-epoch.json
python evo.py --config=configs/toy/ps-sup-toy.json
python evo.py --config=configs/toy/noshare-sup-toy-e1d1.json
```

In [ ]:
!python evo.py --config=configs/toy/sup-toy.json --local

In [ ]:
!python evo.py --config=configs/toy/sup-toy-epoch.json --local

In [ ]:
!python evo.py --config=configs/toy/ps-sup-toy.json --local

## pooling source-side embeddings

In [ ]:
import torch

input_ids = torch.tensor([
    [1, 2, 3, 2],
    [0, 1, 4, 1]
])

mem = torch.tensor([
    [[1.0, 2.0, 3.0],
     [4.0, 5.0, 6.0],
     [7.0, 8.0, 9.0],
     [10.0, 11.0, 12.0]],
    
    [[13.0, 14.0, 15.0],
     [16.0, 17.0, 18.0],
     [19.0, 20.0, 21.0],
     [22.0, 23.0, 24.0]]
])

output_ids = torch.tensor([
    [2, 3, 0, 1, 2],
    [1, 4, 2, 0, 1]
])

mask = input_ids[:, :, None] == output_ids[:, None, :]
counts = mask.sum(1)

tgt = (mem.unsqueeze(2) * mask.unsqueeze(-1)).sum(1)

tgt[counts == 0]